In [48]:
#import libraries

import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import hvplot
import hvplot.pandas 
import matplotlib
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
import json

In [49]:
#load .env file
load_dotenv()

True

In [50]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [51]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [52]:
# Request stock symbol from user. This code will work will all available tickers on alpacas.
# "META" ticker has been temporarily hard coded into the system for ease of use

# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")
stock_symbol = "AAPL"

In [53]:
# Establish what time frame increment we will be looking at
# currently we are working with the daily chart, but the theory would be to 
# allow the user to identify what time scale they would like to trade on
# shorter time frame allows for "day trades" larger time scales are geared
# more towards longer term investments and swing trading.

time_frame = "1Day"

In [54]:
# Get user input for the start date in datetime format
# the start date has been hard coded for ease of programming and troubleshooting our code

# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2010-10-10", tz="America/New_York").isoformat()

In [55]:
# Get user input for the end date in datetime format
# the end date has been hard coded for ease of programming and troubleshooting our code

# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2019-10-10", tz="America/New_York").isoformat()

In [56]:
# trading_days = end_date - start_date

# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [57]:
# Calls the stock data from the period of time desired in the desired time increment

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [58]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 950 entries, 2016-01-04 05:00:00+00:00 to 2019-10-10 04:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        950 non-null    float64
 1   high         950 non-null    float64
 2   low          950 non-null    float64
 3   trade_count  950 non-null    int64  
 4   open         950 non-null    float64
 5   volume       950 non-null    int64  
 6   vwap         950 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 59.4 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,351452,102.61,71935339,104.091749
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,321365,105.75,58690536,103.400868
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,409164,100.56,71079827,100.843959
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,462836,98.68,85996453,98.011371
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,419555,98.55,75066292,97.897369


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2019-10-04 04:00:00+00:00,227.01,227.49,223.89,312401,225.64,36684452,225.940188
2019-10-07 04:00:00+00:00,227.06,229.93,225.84,322412,226.27,32128423,228.049398
2019-10-08 04:00:00+00:00,224.40,228.06,224.33,351143,225.82,31002549,225.827520
2019-10-09 04:00:00+00:00,227.03,227.79,225.64,206603,227.03,20225311,226.781530
2019-10-10 04:00:00+00:00,230.09,230.44,227.30,308883,227.93,30099550,229.620502


In [59]:
# Reset index and display the first 5
# this also prepares for use with plotly library/visualization

#stock_data_plot = stock_data.reset_index()

# clean up datafram by removing vwap, volume, trade count columns
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results

stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
stock_data_plot = stock_data.reset_index()

stock_data.head()

,close,high,low,open
timestamp,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55


In [60]:
# Assign plot values to plotly

fig = go.Candlestick(x=stock_data_plot['timestamp'],
                open=stock_data_plot['open'],
                high=stock_data_plot['high'],
                low=stock_data_plot['low'],
                close=stock_data_plot['close'])


cand = go.Figure(data=[fig])

In [61]:
# Customize the figure (optional)

cand.update_layout(
    width=1200, height=800,
    title="AAPL, OCT - DEC",
    yaxis_title='AAPL Stock'
)


In [62]:
# Add new data columns to DF for body size and candle size
# body size and direction is value of open - close. 
# candle size is the absolute value of low - high

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

# Display columns with added columns

stock_data.head()

,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61,2.74,3.368
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75,-3.04,3.440
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56,0.14,2.500
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68,-2.23,3.700
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55,-1.59,2.350


In [63]:
# initialize variables to be used in calculations
# identify the max body size and minimum candle size within the dataframe

max_body_size = stock_data["body_size"].max()
min_candle_size = stock_data["candle_size"].min()

# keeps track of the count which would initiate a buy or sell
# current count is initialized at 0 as the count is calculated based off of historical data

current_count = 0

# the threshold that would initiate a buy and or sell

threshold = 2

# adds a documenting column to the dataframe and assigns a flat value to all rows

stock_data["card_count"] = current_count

# identifies the number of rows to be used to break the for loop

number_of_rows = len(stock_data)

# print information up to date to identify data progress

# print(max_body_size)
# print(min_candle_size)
# print(previous_candle)
# print(stock_data.iloc[0][4])
# print(number_of_rows)
# stock_data.head()

In [64]:
# ...::: working code :::...
# initialize variables and dataframe for tracking buys, sells, and profits

initial_investment = 10000

holding_binary = 0

# initialize strategy tracking variables and add to stock_data df.
stock_data["capital"] = initial_investment
stock_data["number_of_shares"] = 0
stock_data["investment"] = 0
stock_data["profits"] = 0

capital = initial_investment
number_of_shares = 0
investment = 0
#investment_2 = 0
profits = 0

stock_data.head()

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61,2.74,3.368,0,10000,0,0,0
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75,-3.04,3.440,0,10000,0,0,0
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56,0.14,2.500,0,10000,0,0,0
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68,-2.23,3.700,0,10000,0,0,0
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55,-1.59,2.350,0,10000,0,0,0


In [65]:
# initialize variables and assign values based off of first row for calculations

previous_close = stock_data.iloc[0][0]
previous_high = stock_data.iloc[0][1]
previous_low = stock_data.iloc[0][2]
previous_open = stock_data.iloc[0][3]
previous_body_size = stock_data.iloc[0][4]
previous_candle_size = stock_data.iloc[0][5]

# begin for loop to start calculating each row's data to assign a +1 or -1
# to our count and add adjust our current/previous counts accordingly

for index, row in islice(stock_data.iterrows(), 1, None):

# create an if statement to make sure the for loop finishes at the end of the data appropriately  
    
    if index == (number_of_rows):
        break
    
# assign current day numbers to work with conditional statements

    current_close = row[0]
    current_high = row[1]
    current_low = row[2]
    current_open = row[3]
    current_body_size = row[4]
    current_candle_size = row[5]
    # capital = row[7]
    # number_of_shares = row[8]
    # investment = row[9]
    # profits = row[10]

   # begin of conditional statements. this portion identifies what values to assign
   # to each individal row's data and to determine whether a threshold of "sell" or "buy"
   # is reached. this portion can be adjusted/added to in order to take into consideration
   # additional bar patterns

    # bullish hammer
    if (current_body_size > 0) and (current_body_size*2) < (current_candle_size):
        current_count = current_count + 1
    # bearish hammer
    elif (current_body_size < 0) and ((np.absolute(current_body_size)*2) < (current_candle_size)):
        current_count = current_count - 1
    # bullish engulfing candle
    elif (current_body_size > 0) and (previous_body_size < 1) and (current_body_size > previous_body_size):
        current_count = current_count + 1
    # bearish engulfing candle
    elif (current_body_size < 0) and (previous_body_size > 1) and (np.absolute(current_body_size) > (previous_body_size)):
        current_count = current_count - 1
    # bullish doji candle
    #elif (previous_body_size < 0) and (np.absolute(current_body_size) < (current_close * max_body_size)):
    #    current_count = current_count + 1

    # contains the current count actual value to the desired "buy/sell" threshold

    if current_count > threshold:
        current_count = threshold
    elif current_count < -(threshold):
        current_count = -(threshold)

    # conditional statements to identify when to purchase or sell a stock

    if (current_count == threshold) and (number_of_shares == 0):
        number_of_shares = np.floor(capital / current_close)
        investment = np.floor(capital / current_close) * current_close
        capital = capital - investment
    elif (current_count == -threshold) and (number_of_shares != 0):
        profits = (number_of_shares * current_close) - investment
        capital = capital + investment + profits
        investment = 0
        number_of_shares = 0

        #stock_data.at[index,"investment"] = number_of_shares * current_close
        #profits = row[9] - investment
        #stock_data.at[index,"profits"] = profits

    stock_data.at[index,"number_of_shares"] = number_of_shares
    stock_data.at[index,"investment"] = investment
    stock_data.at[index,"profits"] = profits
    stock_data.at[index,"capital"] = capital
    # set variables to prepare to analyze the following row

    previous_close = row[0]
    previous_high = row[1]
    previous_low = row[2]
    previous_open = row[3]
    previous_body_size = row[4]
    previous_candle_size = row[5]

    # keep track of the current count within the dataframe. assign that
    # to the current working row
    
    stock_data.at[index,"card_count"] = current_count
    
display(stock_data.head(40))
display(stock_data.tail(40))





,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.3680,102.0000,102.61,2.74,3.3680,0,10000.0,0,0.0,0.0
2016-01-05 05:00:00+00:00,102.71,105.8500,102.4100,105.75,-3.04,3.4400,-1,10000.0,0,0.0,0.0
2016-01-06 05:00:00+00:00,100.70,102.3700,99.8700,100.56,0.14,2.5000,0,10000.0,0,0.0,0.0
2016-01-07 05:00:00+00:00,96.45,100.1300,96.4300,98.68,-2.23,3.7000,0,10000.0,0,0.0,0.0
2016-01-08 05:00:00+00:00,96.96,99.1100,96.7600,98.55,-1.59,2.3500,0,10000.0,0,0.0,0.0
2016-01-11 05:00:00+00:00,98.53,99.0600,97.3400,98.97,-0.44,1.7200,-1,10000.0,0,0.0,0.0
2016-01-12 05:00:00+00:00,99.96,100.6900,98.8399,100.55,-0.59,1.8501,-2,10000.0,0,0.0,0.0
2016-01-13 05:00:00+00:00,97.39,101.1900,97.3000,100.32,-2.93,3.8900,-2,10000.0,0,0.0,0.0
2016-01-14 05:00:00+00:00,99.52,100.4800,95.7400,97.96,1.56,4.7400,-1,10000.0,0,0.0,0.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2019-08-15 04:00:00+00:00,201.740,205.1400,199.6700,203.46,-1.720,5.4700,0,67.486,85,16868.25,-3616.65
2019-08-16 04:00:00+00:00,206.500,207.1600,203.8400,204.28,2.220,3.3200,1,67.486,85,16868.25,-3616.65
2019-08-19 04:00:00+00:00,210.350,212.7307,210.0250,210.62,-0.270,2.7057,0,67.486,85,16868.25,-3616.65
2019-08-20 04:00:00+00:00,210.360,213.3500,210.3200,210.88,-0.520,3.0300,-1,67.486,85,16868.25,-3616.65
2019-08-21 04:00:00+00:00,212.640,213.6500,211.6032,212.99,-0.350,2.0468,-2,18141.886,0,0.00,1206.15
2019-08-22 04:00:00+00:00,212.460,214.4350,210.7500,213.19,-0.730,3.6850,-2,18141.886,0,0.00,1206.15
2019-08-23 04:00:00+00:00,202.640,212.0510,201.0000,209.43,-6.790,11.0510,-2,18141.886,0,0.00,1206.15
2019-08-26 04:00:00+00:00,206.490,207.1900,205.0573,205.86,0.630,2.1327,-1,18141.886,0,0.00,1206.15
2019-08-27 04:00:00+00:00,204.160,208.5500,203.5300,207.86,-3.700,5.0200,-1,18141.886,0,0.00,1206.15


In [67]:
# Plot Line graph for the profits

line_plot = stock_data.hvplot.line(
    x='timestamp', 
    y='profits',
)



In [69]:
line_plot

:Curve   [timestamp]   (profits)

In [ ]:
# Getting total profits

stock_data['Total_Investment'] = stock_data['capital']

In [ ]:
# # Initialize other variables
# amount = initial_investment
# number_of_shares = 0  # No initial position
# investment_bought = 0
# profits = 0

# # Define a list of threshold values for buy and sell
# threshold = 2  # Adjust this threshold as needed
# start_date = pd.Timestamp("2020-10-10", tz="America/New_York").isoformat()
# end_date = pd.Timestamp("2020-12-12", tz="America/New_York").isoformat()
# # Calls the stock data from the period of time desired in the desired time increment
# stock_data = api.get_bars(
#     stock_symbol, 
#     time_frame, 
#     start = start_date, 
#     end = end_date
# ).df
# # Drop columns which aren't needed(trade_count, volume, vwap) and display the results
# stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
# display(stock_data.head())
# # Define a list of dates (assuming you have a date for each trading day)
# dates = ["2020-10-10", "2020-10-12", "2020-11-03", "2020-11-04", "2020-12-12"]
# thresholds = [2, -3, 0, 1, -2]  # Example threshold values for each day
# # Iterate through each trading day
# for i in range(len(dates)):
#     date = dates[i]
#     threshold_value = thresholds[i]
#     # Check if you can buy or sell based on the threshold
#     if threshold_value > 2 and number_of_shares == 0:
#         # Buy condition: Invest all the available amount
#         investment_bought = amount
#         number_of_shares = amount
#         amount = 0  # No cash remaining
    
#     elif threshold_value < -2 and number_of_shares > 0:
#         # Sell condition: Sell all the shares you have
#         amount += number_of_shares
#         profits += amount - investment_bought
#         investment_bought = 0
#         number_of_shares = 0
#          # Update the DataFrame with the trading day results
#   # Update the DataFrame with the trading day results
# strategy_df.loc[i] = {
#     "Date": date,
#     "Initial_Investment": initial_investment,
#     "Amount": amount,
#     "Number_of_Shares": number_of_shares,
#     "Investment_Bought": investment_bought,
#     "Profits": profits
# }